# テーブルの結合 (groupby, agg, merge)

In [ ]:
!git clone https://github.com/nejumi/fe_workshop.git

In [1]:
import numpy as np
import scipy as sp
from scipy import optimize
import pandas as pd 
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 100)

In [ ]:
# collaboratoryにLendingClub50000.csv, spx2.csvをアップロードする。
from google.colab import files
#uploaded = files.upload()

In [2]:
# ファイルを読み込む
df = pd.read_csv('dataset/LendingClub50000.csv')
spx = pd.read_csv('dataset/spx2.csv', parse_dates=['date'])

In [3]:
df.head()

,ID,member_id,year,month,loan_amount,purpose_type,purpose_detail,title,emp_length,home_ownership,annual_inc,zip_code,addr_state,dti,grade,sub_grade,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,bad_loan
0,1529851,1793711,2012,9,16000.0,debt_consolidation,Debt Consolidation / Final Wedding Exp,PwC,3.0,RENT,90400.0,080xx,NJ,23.72,A,A4,0.0,Oct-2003,0.0,NaN,NaN,21.0,0.0,3946.0,22.4,42.0,f,0.0,NaN,INDIVIDUAL,0.0,0.0,164787.0,False
1,1824764,2126933,2012,11,3600.0,debt_consolidation,Crush Credit Cards,Morgan Stanley Smith Barney,2.0,RENT,42500.0,217xx,MD,17.34,B,B3,0.0,Dec-2004,0.0,NaN,NaN,16.0,0.0,10927.0,63.5,30.0,f,0.0,NaN,INDIVIDUAL,0.0,8806.0,96627.0,False
2,403548,442721,2009,5,19200.0,wedding,Consolidate debt and pay for wedding,Aggregate Knowledge,1.0,RENT,95000.0,940xx,CA,7.58,B,B4,0.0,Mar-1999,0.0,NaN,NaN,11.0,0.0,14006.0,20.3,26.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False
3,646411,799671,2011,1,21000.0,credit_card,Wells BofA Credit Card Refi,Emerson Process Management,1.0,RENT,85000.0,926xx,CA,17.07,B,B5,0.0,Jun-1999,1.0,NaN,NaN,12.0,0.0,36280.0,47.6,25.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False
4,552526,711946,2010,7,4000.0,other,Debt Consolidation,Home Depot,1.0,RENT,63500.0,303xx,GA,19.50,C,C4,1.0,Oct-2002,2.0,23.0,NaN,16.0,0.0,595.0,11.4,24.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False


In [4]:
spx.iloc[:10]

,date,close
0,1986-01-02,209.59
1,1986-01-03,210.88
2,1986-01-06,210.65
3,1986-01-07,213.80
4,1986-01-08,207.97
5,1986-01-09,206.11
6,1986-01-10,205.96
7,1986-01-13,206.72
8,1986-01-14,206.64
9,1986-01-15,208.26


In [5]:
# 年と月を日付から抽出する
spx['year'] = spx.date.dt.year
spx['month'] = spx.date.dt.month

In [6]:
spx.head()

,date,close,year,month
0,1986-01-02,209.59,1986,1
1,1986-01-03,210.88,1986,1
2,1986-01-06,210.65,1986,1
3,1986-01-07,213.80,1986,1
4,1986-01-08,207.97,1986,1


In [7]:
# 年と月でgroupbyしてそれぞれS&P終値の統計値を算出する
summary = spx.groupby(['year', 'month']).agg(['mean', 'min', 'max'])
summary.columns = ["_".join(x) for x in summary.columns.ravel()]
summary.reset_index(inplace=True)

summary.head()

,year,month,close_mean,close_min,close_max
0,1986,1,208.194091,203.49,213.80
1,1986,2,219.365263,212.79,226.92
2,1986,3,232.327000,224.34,238.97
3,1986,4,237.967727,228.63,244.74
4,1986,5,238.459048,232.76,247.98


In [8]:
# ローンデータとマージする
df = df.merge(summary, on=['year', 'month'], how='left')

In [9]:
df.head()

,ID,member_id,year,month,loan_amount,purpose_type,purpose_detail,title,emp_length,home_ownership,annual_inc,zip_code,addr_state,dti,grade,sub_grade,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,bad_loan,close_mean,close_min,close_max
0,1529851,1793711,2012,9,16000.0,debt_consolidation,Debt Consolidation / Final Wedding Exp,PwC,3.0,RENT,90400.0,080xx,NJ,23.72,A,A4,0.0,Oct-2003,0.0,NaN,NaN,21.0,0.0,3946.0,22.4,42.0,f,0.0,NaN,INDIVIDUAL,0.0,0.0,164787.0,False,1443.419474,1403.44,1465.77
1,1824764,2126933,2012,11,3600.0,debt_consolidation,Crush Credit Cards,Morgan Stanley Smith Barney,2.0,RENT,42500.0,217xx,MD,17.34,B,B3,0.0,Dec-2004,0.0,NaN,NaN,16.0,0.0,10927.0,63.5,30.0,f,0.0,NaN,INDIVIDUAL,0.0,8806.0,96627.0,False,1394.512381,1353.33,1428.39
2,403548,442721,2009,5,19200.0,wedding,Consolidate debt and pay for wedding,Aggregate Knowledge,1.0,RENT,95000.0,940xx,CA,7.58,B,B4,0.0,Mar-1999,0.0,NaN,NaN,11.0,0.0,14006.0,20.3,26.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,902.408500,877.52,929.23
3,646411,799671,2011,1,21000.0,credit_card,Wells BofA Credit Card Refi,Emerson Process Management,1.0,RENT,85000.0,926xx,CA,17.07,B,B5,0.0,Jun-1999,1.0,NaN,NaN,12.0,0.0,36280.0,47.6,25.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,1282.618500,1269.75,1299.54
4,552526,711946,2010,7,4000.0,other,Debt Consolidation,Home Depot,1.0,RENT,63500.0,303xx,GA,19.50,C,C4,1.0,Oct-2002,2.0,23.0,NaN,16.0,0.0,595.0,11.4,24.0,f,0.0,NaN,INDIVIDUAL,0.0,NaN,NaN,False,1079.803333,1022.58,1115.01


In [10]:
df.to_csv('LendingClub50000_spx.csv', index=False)

In [ ]:
# ダウンロードしたら、DataRobotでもう一度予測してみよう
files.download('LendingClub50000_spx.csv')